In [1]:
import cv2
import glob
import numpy as np
import os
import json
import pickle

import Utils.Data.DataGen as DataGen
import Utils.Data.DataLoading as DataLoading
import Utils.PTModel.Inference as Inference

MODELNAME = "TestDropLRVAE2"

In [2]:
# Code to generate fixed lode runner (16 x 16 tile instead of 8 x 8) levels using my resizing solution
# If the path "../data/vglc/Lode Runner/Original_Fixed_Tom" already contains 150 lode runner levels don't run the code
# If the levels are not there make sure the folder "../data/vglc/Lode Runner/Original_Fixed_Tom" is deleted then run this block of code

if not os.path.isdir("../data/vglc/Lode Runner/Original_Fixed_Tom"):
    os.mkdir("../data/vglc/Lode Runner/Original_Fixed_Tom")

    lrLevelPaths = sorted(glob.glob("../data/vglc/Lode Runner/Processed/*.txt"))
    lrLevelStrings = []

    for levelPath in lrLevelPaths:
        with open(levelPath, "r") as f:
            levelString = np.array([list(line[:-1]) for line in f.readlines()])
            
        DataLoading.TextTileToImage(levelString, 16, "../data/tomData/sprites/loderunner", savePath="../data/vglc/Lode Runner/Original_Fixed_Tom", fileNameOverride=levelPath[35:-4], cvtColor=False)

In [3]:
# Gets lode runner level images and adds 16 x 16 border around the edge then splits into 3x3 kernels
lrLevelPaths = sorted(glob.glob("../data/vglc/Lode Runner/Original_Fixed_Tom/*.png"))
lrLevels = [cv2.copyMakeBorder(cv2.cvtColor(cv2.imread(levelPath), cv2.COLOR_BGR2RGB), 16, 16, 16, 16, cv2.BORDER_CONSTANT) for levelPath in lrLevelPaths]
lrLevelKernals = [DataGen.ResizeLevel(level, 16, 3, columnWise=False) for level in lrLevels]

In [4]:
# Gets affordances for all lode runner levels in correct format and layout
lrStringLevelPaths = sorted(glob.glob("../data/vglc/Lode Runner/Processed/*.txt"))

with open("../data/vglc/Lode Runner/Loderunner.json", "r") as f:
    affordanceJson = json.load(f)["tiles"]

with open("../data/tomData/AffordanceTrainDataDictionary.pickle", "rb") as f:
    affordanceDict = pickle.load(f)

levelStrings = []
lrLevelEncodedAffordances = []

for i, levelStringPath in enumerate(lrStringLevelPaths):
    with open(levelStringPath, "r") as f:
        levelString = np.array([list(line[:-1]) for line in f.readlines()])
    levelString = np.pad(levelString, 1, constant_values='.')
    
    lrStringLevelKernals = DataGen.ResizeLevel(levelString, 1, 3, columnWise=False)
    lrStringAffordances = [DataGen.GetAffordances(kernel, affordanceJson, True) for kernel in lrStringLevelKernals]

    lrLevelEncodedAffordances.append([np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in affordanceList]), axis=0) for affordanceList in lrStringAffordances])

lrLevelEncodedAffordances = np.array(lrLevelEncodedAffordances)

In [6]:
Inference.SaveLevelUnifiedRepresentation(None, lrLevelKernals, MODELNAME, "loderunner", affordanceData=lrLevelEncodedAffordances)